<a href="https://colab.research.google.com/github/erelin6613/ML/blob/master/google_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding, Flatten, SimpleRNN, Reshape
import gensim.downloader as api
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [0]:
def cat_to_numeric(category):
    if category=='LIFE_ARTS':
        return 1
    if category=='CULTURE':
        return 2
    if category=='SCIENCE':
        return 3
    if category=='STACKOVERFLOW':
      return 4
    if category=='TECHNOLOGY':
      return 5

In [0]:
train_data = pd.read_csv('/content/train.csv', encoding='utf-8')
final_test_data = pd.read_csv('/content/test.csv', encoding='utf-8')

to_drop_train = ['question_user_page', 'host', 'url', 'answer_user_page', 
                 'answer_user_name', 'question_user_name', 'question_user_page']
#train_data.set_index('qa_id', inplace=True)
train_data = train_data.drop(to_drop_train, axis=1)
#print(train_data['category'].unique())
train_data['categoty_normalized'] = train_data['category'].apply(cat_to_numeric)
print(train_data.columns)
#print(train_data['categoty_normalized'])
train_data.drop('category', axis=1, inplace=True)
#train_data['category_encoded'] = to_categorical(train_data['categoty_normalized'],
#                                  num_classes=5)
train_cols = final_test_data.columns
target_cols = set(train_data.columns).difference(set(final_test_data.columns))
train_cols = set(train_data.columns) - target_cols
target_cols = list(target_cols)
train_cols = list(train_cols)

print(train_cols)
print(target_cols)

Index(['qa_id', 'question_title', 'question_body', 'answer', 'category',
       'question_asker_intent_understanding', 'question_body_critical',
       'question_conversational', 'question_expect_short_answer',
       'question_fact_seeking', 'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'ans

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train_data.loc[:, train_cols], train_data.loc[:, target_cols], test_size=0.25)
text_cols = ['question_title', 'question_body', 'answer']
tokenizer = Tokenizer()
for col in text_cols:
  tokenizer.fit_on_texts(train_data[col])
print(X_train.columns)
for col in text_cols:
  X_train[col+'_tokenized'] = tokenizer.texts_to_sequences(X_train[col])
  X_train.drop([col], inplace=True, axis=1)
  X_test[col+'_tokenized'] = tokenizer.texts_to_sequences(X_test[col])
  X_test.drop([col], inplace=True, axis=1)
  #X_train[col+'_normalized'] = pad_sequences(X_train[col+'_tokenized'], maxlen=750)
  #print(X_train[col+'_tokenized'])
"""
for col in text_cols:
  X_test.loc[:, col+'_tokenized'] = pd.Series([arr for arr in X_train[col+'_tokenized']])
  X_test.loc[:, col+'_tokenized'] = pd.Series(pad_sequences(X_train[col+'_tokenized'], maxlen=750, padding='post'))
  #arr_question_tittle = [arr for arr in X_train['question_title_tokenized']]
  #pads_question_tittle = pad_sequences(arr_question_tittle, maxlen=750, padding='post')
  print(X_test.loc[:, col+'_tokenized'])

for column, row in X_train.iterrows():
  print(pad_sequences([row[text_cols[0]+'_tokenized'], row[text_cols[1]+'_tokenized'], row[text_cols[2]+'_tokenized']], maxlen=700))
  X_train.loc[row['qa_id'], col+'_normalized'] =  pad_sequences([row[text_cols[0]+'_tokenized'], row[text_cols[1]+'_tokenized'], row[text_cols[2]+'_tokenized']], maxlen=700)
  #print(X_train.loc[row['qa_id'], col+'_normalized'])
  break
    #X_train.drop(col+'_tokenized', inplace=True)
"""

print(X_train['question_body_tokenized'])


Index(['question_title', 'qa_id', 'question_body', 'answer'], dtype='object')
1763    [4, 950, 2, 95, 18, 5, 10, 187, 2, 1135, 1, 62...
343     [29, 112, 5, 150, 30, 12, 905, 112, 85, 4547, ...
4926    [66, 2065, 64, 8, 2270, 5628, 1031, 1085, 2700...
3825    [66, 308, 3, 131, 1392, 45, 2045, 25, 600, 166...
5002    [6589, 2996, 715, 29, 357, 279, 16, 1, 28976, ...
                              ...                        
5788    [1677, 1, 1715, 26659, 8, 45, 404, 1492, 5164,...
3754    [4, 63, 2, 17, 227, 2, 49, 216, 2, 1, 337, 425...
3800    [160, 784, 3, 1198, 8, 29, 7747, 6, 32, 253, 2...
3844    [2275, 4, 20, 3, 1382, 484, 9, 4, 25, 23, 132,...
3157    [2275, 4, 20, 3, 112, 9, 160, 144, 796, 75, 13...
Name: question_body_tokenized, Length: 4559, dtype: object


In [0]:
#word_vectors = api.load("glove-wiki-gigaword-100")
#X_train.drop('qa_id', axis=1)
maxlen = 1000
X_train_stacked = []
print(X_train)

for ind in X_train.index:
  items = []
  #print(X_train.loc[ind, 'question_title_tokenized'])
  for col in X_train.drop('qa_id', axis=1).columns:
    if len(X_train.loc[ind, col]) > 1000:
      item_to_stack = X_train.loc[ind, col][:1000]
    else:
      item_to_stack = X_train.loc[ind, col]
      while len(X_train.loc[ind, col]) < 1000:
        item_to_stack.append(0)
    items.append(item_to_stack)

  X_train_stacked.append(np.stack(items))
print(X_train_stacked)

4559


KeyError: ignored

In [0]:
model = Sequential()

model.add(Reshape((None, None, 100), input_shape=X_train.loc[0, :].shape))

#model.add(word_vectors.get_keras_embedding(train_embeddings=True))
model.add(word_vectors.get_keras_embedding(train_embeddings=True))
#model.add(Flatten())
model.add(Dense(64, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.15))

model.add(Dense(32, activation='relu'))
#model.add(SimpleRNN(30))
model.compile(optimizer='Adam', loss='mean_squared_error')
model.summary()

ValueError: ignored

In [0]:
model.fit(X_train, y_train)
model.save('google_data.model')
#Index(['qa_id', 'question_body', 'category', 'answer', 'question_user_name',
#'answer_user_name', 'question_title'],
#dtype='object')

ValueError: ignored